In [18]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
# from corolization import ResidualEncoder
import torch.nn.functional as F
import numpy as np
from skimage.color import rgb2yuv, yuv2rgb
import matplotlib.pyplot as plt

In [2]:
train_dataset = dsets.CIFAR10(root='./data/',
                               train=True, 
#                                transform=transforms.ToTensor(),
                               download=True)

# plt.imshow(yuv2rgb(rgb2yuv(np.array(train_dataset[0][0]))))
# plt.show()

test_dataset = dsets.CIFAR10(root='./data/',
#                               transform=transforms.ToTensor(),
                              train=False)

Files already downloaded and verified


In [3]:
def massage_dset(dset):
    rtv = []
    for i, (image, _) in enumerate(dset):
        img = rgb2yuv(image)
        rtv.append((np.array(img[:, :, 0:1]).reshape(1,32,32), np.array(img[:, :, 1:]).reshape(2,32,32)))
    
    return rtv

train_dataset = massage_dset(train_dataset)
test_dataset = massage_dset(test_dataset)

In [4]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

In [60]:
encoder = ResidualEncoder(32)

In [61]:
criterion = nn.MSELoss()
learning_rate = 0.001
optimizer = torch.optim.SGD(encoder.parameters(), lr=learning_rate)

In [62]:
for epoch in range(1):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(torch.stack(images))
#         images = Variable(images)
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = encoder.double()(images)
#         outputs = encoder(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

torch.Size([100, 256, 4, 4])
torch.Size([100, 256, 8, 8])


KeyboardInterrupt: 